In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(

    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)
docs = PyPDFLoader("./konproz.pdf").load_and_split(text_splitter = text_splitter ) 


doc_contents = [doc.page_content for doc in docs]

doc_ids = list(range(1, len(docs) + 1))


In [2]:
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer("all-MiniLM-L6-v2", device='cpu')
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device='cpu')

# vectors = model.encode(doc_contents)
# vectors = model.encode(doc_contents, batch_size=12, show_progress_bar=True)
vectors = model.encode(doc_contents, show_progress_bar=True)

/Users/abhishek12.9ipad/miniconda3/envs/konproz/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

atches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 141/141 [04:23<00:00,  1.87s/it]

In [3]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

import streamlit as st


client = QdrantClient(":memory:")

client.recreate_collection(
    collection_name="cf_data",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

client.upload_collection(
    collection_name="cf_data",
    ids=[i for i in range(len(doc_contents))],
    vectors=vectors,
    
)

/var/folders/rq/qqz_mfy94cx5f89cknj2xwqr0000gn/T/ipykernel_76531/2919452441.py:9: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [4]:
from dspy.retrieve.qdrant_rm import QdrantRM
qdrant_retriever_model = QdrantRM("cf_data", client, k=3)


In [5]:
import dspy
lm = dspy.OllamaLocal(model="llama3.2",timeout_s = 180)


dspy.settings.configure(rm=qdrant_retriever_model, lm=lm)


In [24]:
class GenerateAnswer(dspy.Signature):
    """Answer questions by extracting concise, fact based responses from PDF documents."""

    context = dspy.InputField(desc="Extracted text or embeddings from the PDF document")
    question = dspy.InputField(desc="User query related to the content of the PDF")
    answer = dspy.OutputField(desc="Short, factual answer derived from the PDF content")
    # answer = dspy.OutputField(desc="a answer within 20 to 30 words")


def get_context(text):
    query_vector = model.encode(text)


    hits = client.search(
        collection_name="cf_data",
        query_vector=query_vector,
        limit=3  # Return 5 closest points
    )
    s=''
    for x in [doc_contents[i.id] for i in hits]:
        s = s + x
    return s


In [25]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()


        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)


    def forward(self, question):
        context = get_context(question)
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [26]:
rag = RAG()
def respond(query):
    response = rag(query)
    return response.answer

In [28]:
print(respond("Who are not eligible to take Input Tax Credit?"))

A registered person shall be entitled to take credit of input tax charged on any supply of goods or services or both which are used or intended to be used in the course or furtherance of his business, subject to conditions and restrictions prescribed under Section 49.


In [ ]:
Discounts like trade discount, quantity discount etc, are part of the normal trade and commerce. Therefore, pre-supply discounts i.e. discounts recorded in the invoice have been allowed to be excluded while determining the taxable value.